# Revisión de código (Sección 2b y 2c) 
Función a revisar: **calcular_varianza** y **calcular_rendimiento**



In [1]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   7745      0 --:--:-- --:--:-- --:--:--  7745
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
+ pip install -q cupy-cuda100  chainer 
+ set +ex
Installation succeeded!


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import fix_yahoo_finance as yf
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import time

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Código revisado

# 2b. Matriz de varianzas y covarianzas


In [0]:
def calcular_varianza(X):

  """
  Función para calcular el la matriz de varianzas y covarianzas para un conjunto de acciones

  params:
      X      matriz mxn de precios, donde:
             m es el número de observaciones y
               n el número de acciones
  
  return:
      S  matriz de varianzas y covarianzas
  """
  m,n=X.shape
  X = cp.asarray(X)

  X_m = cp.zeros((m-1,n))

  for i in range(n):
    X_m[:,i] = calcular_rendimiento_vector(X[:,i]) - calcular_rendimiento_vector(X[:,i]).mean()

  S = (cp.transpose(X_m)@X_m)/(m-2)

  return S


In [0]:
def calcular_rendimiento_vector(x):
  """
  Función para calcular el rendimiento esperado

  params:
      x     vector de precios
  
  return:
      r_est rendimiento esperado diario
  """

  # Definimos precios iniciales y finales como arreglo alojado en la gpu
  x_o = cp.asarray(x)
  x_f = x_o[1:]

  # Calculamos los rendimientos diarios
  r = cp.log(x_f/x_o[:-1])

  return r

In [0]:
def calcular_rendimiento(X):
  """
  Función para calcular el rendimiento esperado para un conjunto de acciones

  params:
      X      matriz mxn de precios, donde:
             m es el número de observaciones y
             n el número de acciones
  
  return:
      r_est rvector de rendimientos esperados
  """
  m,n = X.shape
  r_est = cp.zeros(n)
  X = cp.asarray(X)

  for i in range(n):
    r_est[i] = calcular_rendimiento_vector(X[:,i]).mean()

  return 264*r_est

**1. Documentacion 2b**

La Documentación expresa de manera clara, consica y breve lo que hace el código. De igual forma se explica de manera clara y concisa los argumentos de entrada y salida. La documentación es completa.


**2. Cumplimiento de objetivos del código 2b**

La función cumple con sus objetivos.

**3. Test 2b**

Revisaremos la función con distintos tamaños de matrices.


In [0]:
np.random.seed(1234)

Matriz de $10 \times 10$


In [10]:
X1 = np.random.uniform(1,1000,10**2).reshape(10,10)
calcular_varianza(X1)
#np.shape(X1)

array([[ 1.46686905,  0.32660669,  1.41717963, -0.83306188, -0.16774303,
         0.16491629,  0.6695301 , -0.17399062, -0.36762837,  0.0910157 ],
       [ 0.32660669,  0.29593255,  0.50620271, -0.7812062 ,  0.09053954,
         0.03904355,  0.60888277,  0.34826037,  0.21496537,  0.08293203],
       [ 1.41717963,  0.50620271,  3.15870697, -1.966857  , -0.17401216,
         0.10591102,  1.45355125, -0.87745901,  1.35280437,  0.64724087],
       [-0.83306188, -0.7812062 , -1.966857  ,  4.71328479, -0.01489054,
        -2.05924445, -1.94686907, -0.57410833, -0.3589756 , -1.11079972],
       [-0.16774303,  0.09053954, -0.17401216, -0.01489054,  0.42285194,
        -0.3588606 ,  0.12582499,  0.93943973,  0.94457383,  0.03144309],
       [ 0.16491629,  0.03904355,  0.10591102, -2.05924445, -0.3588606 ,
         2.65690429,  0.27565966, -0.53696766, -1.56913399,  1.0138904 ],
       [ 0.6695301 ,  0.60888277,  1.45355125, -1.94686907,  0.12582499,
         0.27565966,  1.54412462, -0.08367778

In [11]:
np.shape(X1)

(10, 10)

Matriz de $10^3 \times 10$


In [12]:
X2 = np.random.uniform(1,1000,10**4).reshape(10**3,10**1)
calcular_varianza(X2)

array([[ 2.2220343 ,  0.01430896,  0.01414881, -0.01596572, -0.01060321,
         0.01307148, -0.06304303, -0.00858486, -0.11306426, -0.06876012],
       [ 0.01430896,  1.82276294,  0.01366593,  0.00967094,  0.05086246,
         0.12828126, -0.04608573,  0.09117425,  0.01282499,  0.07321494],
       [ 0.01414881,  0.01366593,  2.00522958,  0.07933775, -0.02135849,
         0.05610349,  0.05754015, -0.06438416,  0.00281881,  0.01330553],
       [-0.01596572,  0.00967094,  0.07933775,  1.89751426, -0.00484842,
         0.05820801, -0.03193703,  0.01016307,  0.17865124,  0.08838178],
       [-0.01060321,  0.05086246, -0.02135849, -0.00484842,  2.0359547 ,
         0.08282926, -0.03087633, -0.10618265,  0.10671397,  0.07563415],
       [ 0.01307148,  0.12828126,  0.05610349,  0.05820801,  0.08282926,
         1.87854141, -0.0069636 ,  0.00470695,  0.08336298, -0.11552537],
       [-0.06304303, -0.04608573,  0.05754015, -0.03193703, -0.03087633,
        -0.0069636 ,  1.86862565,  0.0317846 

In [13]:
np.shape(X2)

(1000, 10)

Matriz de $10^5 \times 10^2$


In [14]:
X3 = np.random.uniform(1,1000,10**7).reshape(10**5,10**2)
calcular_varianza(X3)

array([[ 1.90773425e+00, -4.74094740e-03, -6.52402508e-04, ...,
         4.62774843e-03,  2.24041285e-03, -5.52823819e-03],
       [-4.74094740e-03,  1.90521430e+00,  5.63837585e-03, ...,
        -3.07097586e-03, -2.07468383e-03,  6.74957800e-04],
       [-6.52402508e-04,  5.63837585e-03,  1.88388822e+00, ...,
        -7.69746875e-03, -7.71617273e-03,  1.23158279e-02],
       ...,
       [ 4.62774843e-03, -3.07097586e-03, -7.69746875e-03, ...,
         1.89130400e+00,  2.57514930e-04,  4.44636776e-04],
       [ 2.24041285e-03, -2.07468383e-03, -7.71617273e-03, ...,
         2.57514930e-04,  1.90559587e+00,  3.62411502e-03],
       [-5.52823819e-03,  6.74957800e-04,  1.23158279e-02, ...,
         4.44636776e-04,  3.62411502e-03,  1.89037822e+00]])

In [15]:
np.shape(X3)

(100000, 100)

Matriz de $10^6 \times 10^2$

In [16]:
X4 = np.random.uniform(1,1000,10**8).reshape(10**6,10**2)
calcular_varianza(X4)

array([[ 1.90844005e+00,  4.38419757e-04, -5.84674553e-03, ...,
         1.78756058e-03,  4.19257513e-04,  2.67240522e-03],
       [ 4.38419757e-04,  1.90074785e+00,  3.40623892e-03, ...,
         3.13311333e-03,  4.21853388e-04, -2.77607499e-03],
       [-5.84674553e-03,  3.40623892e-03,  1.89996730e+00, ...,
        -3.43282584e-03,  4.32868359e-03, -1.26973465e-03],
       ...,
       [ 1.78756058e-03,  3.13311333e-03, -3.43282584e-03, ...,
         1.90014121e+00,  1.76250355e-03, -3.17588156e-05],
       [ 4.19257513e-04,  4.21853388e-04,  4.32868359e-03, ...,
         1.76250355e-03,  1.90808714e+00, -2.36167338e-04],
       [ 2.67240522e-03, -2.77607499e-03, -1.26973465e-03, ...,
        -3.17588156e-05, -2.36167338e-04,  1.91068095e+00]])

In [17]:
np.shape(X4)

(1000000, 100)

Principales hallazos del test 1:

* La función trabaja de manera distintos tamaños de matriz.

# 2c. Valor medio esperado de activos

**Test 2**

**Objetivo del test:** Verificar que la función del valor medio esperado funcione correctamente para distintos tamaños de acciones.

**Implementación del test:**

**1. Documentacion 2c**

La Documentación expresa de manera clara, consica y breve lo que hace el código. De igual forma se explica de manera clara y concisa los argumentos de entrada y salida. La documentación es completa.


**2. Cumplimiento de objetivos del código 2c**

La función cumple con sus objetivos.

**3. Test 2c**

Revisaremos la función con distintos tamaños de matrices utilizados en el test anterior.

In [51]:
calcular_rendimiento(X1)

array([ -6.62995855,  -2.6435459 ,   5.50096451,   5.62033096,
       -14.19526978,  17.89509847,  19.4168558 ,   0.62641881,
       -82.23850534,  -7.87667006])

In [52]:
calcular_rendimiento(X2)

array([-0.12666371, -0.49415459, -0.55033777,  0.06097676, -0.22545761,
        0.38782181,  0.16931582, -0.14137867,  0.36653942, -1.4634766 ])

In [54]:
calcular_rendimiento(X3)

array([ 0.00437884, -0.00342554, -0.00398738,  0.00172272, -0.00367318,
       -0.001502  , -0.00036215, -0.00424283,  0.00010382,  0.00283781,
        0.00379868,  0.00064378, -0.0019029 ,  0.00113643, -0.00492621,
        0.00024626,  0.00487415, -0.00066194, -0.00336709,  0.00124379,
        0.01070217, -0.00099479,  0.00407496,  0.00185662, -0.00412237,
        0.0017609 , -0.00262141, -0.00227729,  0.00793163, -0.00319564,
       -0.00010678, -0.00293843, -0.01194529, -0.00144341, -0.00478833,
        0.00044674, -0.0041599 , -0.00075774,  0.00463505,  0.00336376,
       -0.00046149, -0.00025119,  0.00018817,  0.00055234, -0.00425386,
        0.0079367 , -0.0004536 , -0.00189483,  0.00387737,  0.0007939 ,
       -0.0012152 ,  0.00267712,  0.00391974, -0.00202013,  0.00108686,
        0.00104914, -0.00080633,  0.00122853,  0.00384108, -0.00166969,
       -0.0013335 ,  0.00392317,  0.00016839, -0.00542378, -0.00102632,
       -0.00412347,  0.00248822,  0.00319685, -0.00043782,  0.00

Principales hallazos del test 2:

* El código funciona de manera correcta para distintos tamaños de matrices de acciones.

Las funciones trabajan de manera correcta, no se encontraron errores en la implementación.